In [2]:
def generate_best(n):
    L=generate_n(n)
    r=sorted(L, key=lambda x: x[1], reverse=True)
    print(r[0])
    
import random

def get_generation_by_gram(grammar_str: str, target, stmt_split='=', or_split='|'):
    rules = dict()  # key is the @statement, value is @expression
    for line in grammar_str.split('\n'):
        if not line: continue
        # skip the empty line
        #  print(line)
        stmt, expr = line.split(stmt_split)
        rules[stmt.strip()] = expr.split(or_split)
    generated = generate(rules, target=target)
    return generated


customer_service_grammar = """
    cs => title say_hello 我是 Number 号客服，您想 verb content end
    title => 女士 | 先生 | 小朋友
    say_hello => 早上好！ | 下午好！ |晚上好！
    Number => 1 | 2 |3 |5
    verb =>  了解 | 打印 | 咨询
    content =>  订房业务 | 餐饮业务 | 娱乐活动 |优惠折扣
    end => 吗？"""


def generate(grammar_rule, target):
    if target in grammar_rule: 
        candidates = grammar_rule[target]  
        candidate = random.choice(candidates)  
        return ''.join(generate(grammar_rule, target=c.strip()) for c in candidate.split())
    else:
        return target


get_generation_by_gram(customer_service_grammar, target='cs', stmt_split='=>')


import jieba
corpus = 'article_9k.txt'
FILE = open(corpus).read()
max_length = 1000000 
sub_file = FILE[:max_length]
def cut(string):
    return list(jieba.cut(string))
TOKENS = cut(sub_file)

from collections import Counter
words_count = Counter(TOKENS)
_2_gram_words = [
    TOKENS[i] + TOKENS[i+1] for i in range(len(TOKENS)-1)
]
_2_gram_words[:10]
_2_gram_word_counts = Counter(_2_gram_words)
words_count.most_common()[-1][-1]

def get_1_gram_count(word):
    if word in words_count: return words_count[word]
    else:
        return words_count.most_common()[-1][-1]
def get_2_gram_count(word):
    if word in _2_gram_word_counts: return _2_gram_word_counts[word]
    else:
        return _2_gram_word_counts.most_common()[-1][-1]
def get_gram_count(word, wc):
    if word in wc: return wc[word]
    else:
        return wc.most_common()[-1][-1]
    
def two_gram_model(sentence):
    # 2-gram langauge model
    tokens = cut(sentence)    
    probability = 1 
    for i in range(len(tokens)-1):
        word = tokens[i]
        next_word = tokens[i+1]        
        _two_gram_c = get_gram_count(word+next_word, _2_gram_word_counts)
        _one_gram_c = get_gram_count(next_word, words_count)
        pro =  _two_gram_c / _one_gram_c       
        probability *= pro 
    return probability  




def generate_n(n):
    L=[]
    for i in range(n):
      content=get_generation_by_gram(customer_service_grammar, target='cs', stmt_split='=>')
      pro=two_gram_model(content)
      L.append((content,pro))
    return L

generate_best(5)

('先生早上好！我是1号客服，您想咨询餐饮业务吗？', 4.0063717610968116e-19)
